In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as img
%matplotlib inline
import os
import random
import cv2
from collections import defaultdict
import collections
from shutil import copy
from shutil import copytree, rmtree
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers
from tensorflow.keras import models

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2

In [ ]:
!nvidia-smi

In [ ]:
# Loading the best saved models to make predictions
K.clear_session()
# model_best = load_model('models/EfficientNetV2S/EfficientNetV2S.hdf5', compile=False)
# model_best = load_model('models/EfficientNetV2L/EfficientNetV2L.hdf5', compile=False)
model_best = load_model('/Users/dim__gag/Desktop/EfficientNetV2L/EfficientNetV2L.hdf5', compile=False)

In [ ]:
# Foods sorted
# data_dir = "/workspace/persistent/food-101/images" # This is for DSRI workspace path

data_dir = "/Users/dim__gag/Desktop/food-101/images" # This is for local path

foods_sorted = sorted(os.listdir(data_dir))


# foods_sorted = ['apple_pie','pizza','omelette']

In [ ]:
len(foods_sorted)

In [ ]:
def pick_n_random_classes(n):
  food_list = []
  random_food_indices = random.sample(range(len(foods_sorted)),n) # We are picking n random food classes
  for i in random_food_indices:
    food_list.append(foods_sorted[i])
  food_list.sort()
  return food_list

In [ ]:
food_list = pick_n_random_classes(101)

In [ ]:
food_list

In [ ]:
def predict_class(model, images, show = True):
  for img in images:
    img = image.load_img(img, target_size=(299, 299))
    img = image.img_to_array(img)                    
    img = np.expand_dims(img, axis=0)         
    img /= 255.                                      

    pred = model.predict(img)
    index = np.argmax(pred)
    food_list.sort()
    pred_value = food_list[index]
    if show:
        plt.imshow(img[0])                           
        plt.axis('off')
        plt.title(pred_value) 
        plt.show()

In [ ]:
images = [] 
# images.append('test_images/Apple.jpg')
# images.append('test_images/applepie.jpg')
# images.append('test_images/cupcakes.jpg')
# images.append('test_images/springrolls.jpg')
# images.append('test_images/samosa.jpg')


# images.append('test_images/burger.jpeg')
# images.append('test_images/hardburger.jpg')
# images.append('test_images/dim1.jpeg')
# images.append('test_images/dim2.jpeg')

# images.append('test_images/leo.jpg')


images.append('test_images/steak.jpg')

In [ ]:
%pwd

In [ ]:
images

In [ ]:
# show images
predict_class(model_best, images, show = True)

In [ ]:
wrong = []
wrong.append('test_images/hardburger.jpg')

In [ ]:
predict_class(model_best, wrong, show = True)

## Model Details and Evaluation

In [ ]:
### Model Details

In [ ]:
EfficientNetV2S = load_model('models/EfficientNetV2S/EfficientNetV2S.hdf5')

EfficientNetV2L = load_model('models/EfficientNetV2L/EfficientNetV2L.hdf5')

In [ ]:
def print_model_details(model):
    def namestr(obj, namespace):
        return [name for name in namespace if namespace[name] is obj]
    print("==========================================================")            
    print("Model Name: ", namestr(model, globals()))
    print("Number of Layers: ", len(model.layers))
    print("Number of Parameters: ", format(model.count_params(), ',d'))
    print("==========================================================\n")

In [ ]:
print_model_details(EfficientNetV2S)
print_model_details(EfficientNetV2L)

In [ ]:
'''from collections import defaultdict
import collections
import shutil
import os

# Train Data
classes_images=defaultdict(list)
with open('/Users/dim__gag/Desktop/food-101/meta/train.txt', 'r') as txt:
	paths= [read.strip() for read in txt.readlines()]
	for p in paths:
		food = p.split('/')
		classes_images[food[0]].append(food[1] + '.jpg')

for food in classes_images.keys():
	if not os.path.exists(os.path.join("/Users/dim__gag/Desktop/food-101/train",food)):
		os.makedirs(os.path.join("/Users/dim__gag/Desktop/food-101/train", food))
	for i in classes_images[food]:
		shutil.copyfile(os.path.join("/Users/dim__gag/Desktop/food-101/images", food, i), os.path.join("/Users/dim__gag/Desktop/food-101/train", food, i))
  '''

In [ ]:
'''# Test/Validation Data
classes_images=defaultdict(list)
with open('/Users/dim__gag/Desktop/food-101/meta/test.txt', 'r') as txt:
	paths= [read.strip() for read in txt.readlines()]
	for p in paths:
		food = p.split('/')
		classes_images[food[0]].append(food[1] + '.jpg')

for food in classes_images.keys():
	if not os.path.exists(os.path.join("/Users/dim__gag/Desktop/food-101/test",food)):
		os.makedirs(os.path.join("/Users/dim__gag/Desktop/food-101/test", food))
	for i in classes_images[food]:
		shutil.copyfile(os.path.join("/Users/dim__gag/Desktop/food-101/images", food, i), os.path.join("/Users/dim__gag/Desktop/food-101/test", food, i))'''

In [ ]:
%cd 

In [ ]:
%cd /Users/dim__gag/Desktop/food-101

In [ ]:
# Data Augmentation for Evaluation
train_data_dir = "/Users/dim__gag/Desktop/food-101/train"
validation_data_dir = "/Users/dim__gag/Desktop/food-101/test"
img_width, img_height = 299, 299 # input image dimensions
batch_size = 32 # Change this to your desired batch size
# epochs = 100 # Change the Number of epochs here

def data_augmentation(train_data_dir, validation_data_dir, img_height, img_width, batch_size):
    # Data Augmentation
    train_datagen = ImageDataGenerator(
        rescale=1. / 255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

    test_datagen = ImageDataGenerator(rescale=1. / 255)
    
    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')

    validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')
    return train_generator, validation_generator



train_generator, validation_generator = data_augmentation(train_data_dir=train_data_dir, validation_data_dir=validation_data_dir, img_width=img_height, img_height=img_height, batch_size=batch_size)




In [ ]:
# Print training, test accuracy and loss of the model
def model_eval(model, train, val):
    # evaluate the model
    train_loss, train_acc = model.evaluate(train, verbose=0)
    val_loss, val_acc = model.evaluate(val, verbose=0)
    print('Train loss:', train_loss)
    print('Train accuracy:', train_acc)
    print('Validation loss:', val_loss)
    print('Validation accuracy:', val_acc)

In [ ]:
model_eval(EfficientNetV2S, train_generator, validation_generator)
model_eval(EfficientNetV2L, train_generator, validation_generator)